In [11]:
import pdfplumber
import pandas as pd
import numpy as np

pl_text = '連結損益及び包括利益計算書'
bs_text = '連結貸借対照表'
ic_text = '四半期連結キャッシュ・フロー計算書'

In [8]:
path_file = "/Users/mac/Monthy/Crawl_buffett/tests/Data/1301/PDF/2022_Q2_決算短信(2022_11_4).pdf"
pdf = pdfplumber.open(path_file)

In [12]:
def find_header(pdf,
                page_id: int,):
    page = pdf.pages[page_id]
    text = page.extract_text()
    milion_yen = '百万円'
    if text.find(milion_yen) >= 0:
        text1 = text[text.find(milion_yen) + 5:]
        text2 = text1[:text1.find('\n')]
        text2 = text2.split(' ')
        return [milion_yen, np.nan] + text2
    else:
        return ['N/A', 'N/A', 'N/A', 'N/A']

def extract_data_pdf(pdf,
                     lst_table = [], 
                     page_id = 0,
                     type_data = 'find_keyword', 
                     key_word = str, 
                     n_cols = 4):
    
    page = pdf.pages[page_id]
    tables = page.extract_tables()
    text = page.extract_text()
    if type_data == 'find_keyword':
        find_key = text.find(key_word) >= 0
    if type_data == 'avoid_keyword':
        find_key = text.find(key_word) == -1
    if ((find_key == True) 
        and (len(tables) > 0) 
        and (np.array(tables[0]).shape[1] == n_cols)):
        lst_table += tables[0]
        return lst_table, True
    return lst_table, False

def extract_pdf(path_file):
    pdf = pdfplumber.open(path_file)
    for page_id in range(len(pdf.pages)):
        lst_table, _ = extract_data_pdf(pdf, 
                                        page_id = page_id,
                                        type_data = 'find_keyword',
                                        key_word=pl_text,)
        if _ == True: 
            lst_col = find_header(pdf, 
                                  page_id = page_id)
            print(lst_col)
            break

    while True:
        lst_table, _ = extract_data_pdf(pdf, 
                                        lst_table = lst_table,
                                        page_id = page_id+1,
                                        type_data = 'avoid_keyword',
                                        key_word=bs_text,)
        lst_table, _ = extract_data_pdf(pdf,
                                        lst_table = lst_table,
                                        page_id = page_id+1,
                                        type_data = 'avoid_keyword',
                                        key_word=ic_text,)
        if _ == True:
            page_id += 1
        else: break
    return pd.DataFrame(lst_table, columns = lst_col)

df = extract_pdf(path_file)
df

['百万円', nan, '前第２四半期連結累計期間', '当第２四半期連結累計期間']


,百万円,NaN,前第２四半期連結累計期間,当第２四半期連結累計期間
0,売上高,None,"120,880","133,619"
1,売上原価,None,"106,951","116,774"
2,売上総利益,None,"13,929","16,844"
3,販売費及び一般管理費,None,"10,706","11,469"
4,営業利益,None,"3,222","5,375"
...,...,...,...,...
101,,財務活動によるキャッシュ・フロー,"4,583","15,820"
102,現金及び現金同等物に係る換算差額,None,24,187
103,現金及び現金同等物の増減額（△は減少）,None,△306,△659
104,現金及び現金同等物の期首残高,None,"7,097","6,539"
